<a href="https://colab.research.google.com/github/Selma266/Selma266/blob/main/Malicious-URL-Detection2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Étape: Chargement Phishing URLs dataset + prétraitement

In [ ]:
import pandas as pd #la bibliothèque Pandas, utilisée pour la manipulation et l’analyse de données tabulaires.
df = pd.read_csv("/content/drive/MyDrive/Phishing URLs_cleaned.csv")  # read dataset url,label  #Chargement du dataset
df.head()

,url,label
0,https://docs.google.com/presentation/d/e/2PACX...,malicious
1,https://btttelecommunniccatiion.weeblysite.com/,malicious
2,https://kq0hgp.webwave.dev/,malicious
3,https://brittishtele1bt-69836.getresponsesite....,malicious
4,https://bt-internet-105056.weeblysite.com/,malicious


•	Suppression des lignes redondantes +Supprimer les NaN+ Conversion label en valeur Numeriques

---



In [ ]:
# BEFORE removing duplicates
print("Before removing duplicates:")
print("Shape:", df.shape)

# Step 2: Remove duplicate rows
df = df.drop_duplicates()   #Suppression des doublons

# AFTER removing duplicates
print("\nAfter removing duplicates:")
print("Shape:", df.shape)

Before removing duplicates:
Shape: (504983, 2)

After removing duplicates:
Shape: (504933, 2)


In [ ]:
# Supprimer les lignes où url ou label est NaN  dropna  Supprime les lignes où : l’URL est vide +le label est manquant
df = df.dropna(subset=["url", "label"]).reset_index(drop=True)  #reset_index(drop=True) Réinitialise l’index du DataFrame
# strip() supprime les espaces au début et à la fin URL
df["url"] = df["url"].str.strip()
# Conversion des labels texte en valeurs numériques  astype(int) Force le type entier (int)  Garantit que y contient uniquement 0 ou 1
df["label"] = df["label"].map({"benign": 0, "malicious": 1}).astype(int) #Encodage des labels
print(df["label"].value_counts()) #Vérification de la distribution des classes Afichages

#Supprime les lignes dont l’URL ou le label est manquant, puis réinitialise l’index du DataFrame.

label
0    345738
1    159195
Name: count, dtype: int64


CRÉATION DES 3 DISTRIBUTIONS


Dataset(50% 50%): 88,469 (50 %)classe 0 et 88,469(50 %) classe 1

Dataset(90% 10%): 159,244(90%)classe 0 et 17,694(10%)classe 1

Dataset(10%-90%):17,694(10%)classe 0 , 159,244 (90%)classe 1

"By maintaining a constant dataset size of 176883 samples across all three distributions (50-50, 90-10, and 10-90), we isolated the impact of class imbalance on model performance, ensuring that any observed differences in metrics are attributable solely to the distribution ratio rather than dataset size or other confounding factors."

In [ ]:
import pandas as pd
#  Sépare le dataset en deux sous-ensembles selon la classe (bénigne / malveillante).✅ Correct Séparer bénignes et malveillantes
benign = df[df['label'] == 0]
malicious = df[df['label'] == 1]
#Affiche le nombre d’échantillons par classe pour vérifier la disponibilité des données.
print(f"URLs bénignes disponibles: {len(benign)}")
print(f"URLs malveillantes disponibles: {len(malicious)}")
print(f"Total URLs disponibles: {len(df)}")

# Taille fixe pour tous les datasets
# Adjusted TOTAL_SIZE to avoid 'Cannot take a larger sample than population' error
# Calculation: min(len(malicious) / 0.9, len(benign) / 0.9, len(benign) / 0.5, len(malicious) / 0.5)
# min(159195 / 0.9, 345738 / 0.9, 345738 / 0.5, 159195 / 0.5)
# min(176883.33, 384153.33, 691476, 318390)
TOTAL_SIZE = 176883 # Définit la taille cible commune des trois datasets générés.

# ===== Dataset 1: Équilibré (50-50) =====
def create_balanced_dataset(benign, malicious, total_size=176883):
    """
    Crée un dataset équilibré avec 50% benign et 50% malicious
    """
    size_per_class = total_size // 2  #  176883 // 2 = 88441 pour chaque classe

    ben_sample = benign.sample(n=size_per_class, random_state=42)
    mal_sample = malicious.sample(n=size_per_class, random_state=42)

    dataset = pd.concat([ben_sample, mal_sample]) # Concatène  les deux DataFrames
    dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

    return dataset

# ===== Dataset 2: 90% bénignes, 10% malveillantes =====
def create_imbalanced_dataset_90_10(benign, malicious, total_size=176883):
    """
    Crée un dataset avec 90% benign et 10% malicious
    """
    ben_size = int(total_size * 0.9)  #90% ≈ 159,194
    mal_size = int(total_size * 0.1)  # 10% ≈ 17,688

    ben_sample = benign.sample(n=ben_size, random_state=42)
    mal_sample = malicious.sample(n=mal_size, random_state=42)

    dataset = pd.concat([ben_sample, mal_sample])
    dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

    return dataset

# ===== Dataset 3: 10% bénignes, 90% malveillantes =====
def create_imbalanced_dataset_10_90(benign, malicious, total_size=176883):
    """
    Crée un dataset avec 10% benign et 90% malicious
    """
    ben_size = int(total_size * 0.1)  # 31,849
    mal_size = int(total_size * 0.9)  # 286,641

    ben_sample = benign.sample(n=ben_size, random_state=42)
    mal_sample = malicious.sample(n=mal_size, random_state=42)

    dataset = pd.concat([ben_sample, mal_sample])
    dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

    return dataset

# ===== Créer les 3 datasets =====
print("\n" + "="*60)
print("CRÉATION DES DATASETS")
print("="*60)

dataset_balanced = create_balanced_dataset(benign, malicious, TOTAL_SIZE)
dataset_imb_90_10 = create_imbalanced_dataset_90_10(benign, malicious, TOTAL_SIZE)
dataset_imb_10_90 = create_imbalanced_dataset_10_90(benign, malicious, TOTAL_SIZE)

# ===== Afficher les statistiques =====
print("\n📊 DATASET 1 - BALANCED (50-50):")
print(f"   Total samples: {len(dataset_balanced)}")
print(f"   Distribution:")
print(dataset_balanced['label'].value_counts().sort_index())
print(f"   Pourcentages:")
print(dataset_balanced['label'].value_counts(normalize=True).sort_index() * 100)

print("\n📊 DATASET 2 - IMBALANCED (90% benign - 10% malicious):")
print(f"   Total samples: {len(dataset_imb_90_10)}")
print(f"   Distribution:")
print(dataset_imb_90_10['label'].value_counts().sort_index())
print(f"   Pourcentages:")
print(dataset_imb_90_10['label'].value_counts(normalize=True).sort_index() * 100)

print("\n📊 DATASET 3 - IMBALANCED (10% benign - 90% malicious):")
print(f"   Total samples: {len(dataset_imb_10_90)}")
print(f"   Distribution:")
print(dataset_imb_10_90['label'].value_counts().sort_index())
print(f"   Pourcentages:")
print(dataset_imb_10_90['label'].value_counts(normalize=True).sort_index() * 100)
#Resumer Génère les trois distributions expérimentales à partir du même dataset source.
# value_counts(),value_counts(normalize=True)Affiche le nombre et le pourcentage de chaque classe pour validation finale.

URLs bénignes disponibles: 345738
URLs malveillantes disponibles: 159195
Total URLs disponibles: 504933

CRÉATION DES DATASETS

📊 DATASET 1 - BALANCED (50-50):
   Total samples: 176882
   Distribution:
label
0    88441
1    88441
Name: count, dtype: int64
   Pourcentages:
label
0    50.0
1    50.0
Name: proportion, dtype: float64

📊 DATASET 2 - IMBALANCED (90% benign - 10% malicious):
   Total samples: 176882
   Distribution:
label
0    159194
1     17688
Name: count, dtype: int64
   Pourcentages:
label
0    90.000113
1     9.999887
Name: proportion, dtype: float64

📊 DATASET 3 - IMBALANCED (10% benign - 90% malicious):
   Total samples: 176882
   Distribution:
label
0     17688
1    159194
Name: count, dtype: int64
   Pourcentages:
label
0     9.999887
1    90.000113
Name: proportion, dtype: float64


Tokenization avec RoBERTa
le tokenizer RoBERTa Byte-Level BPE ?
Ce que fait le tokenizer AUTOMATIQUEMENT :

Découpage en sous-mots (subwords)
Ajout des tokens spéciaux [CLS], [SEP]
Padding/truncation
Conversion en IDs numériques
Création des attention masks

In [ ]:
#from transformers import RobertaTokenizer

# Charger le tokenizer
#tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

#print("Tokenizer chargé avec succès!")
#print(f"Taille du vocabulaire: {tokenizer.vocab_size}")
#Ce code charge un tokenizer pré-entraîné qui transforme du texte
# Des URLs  en nombres que les modèles de Deep Learning peuvent comprendre.
#RobertaTokenizer : L'outil qui découpe et convertit le texte en tokens pour le modèle RoBERTa.

Tokenizer chargé avec succès!
Taille du vocabulaire: 50265


In [ ]:
## ÉTAPE 1 : Charger le tokenizer
import torch #Importe PyTorch, utilisé pour la gestion des tenseurs, du GPU/CPU et des calculs de deep learning.
#Rôle : Importe PyTorch, la bibliothèque de deep learning,Utilité : Gère les tenseurs, GPU, et calculs mathématiques
!pip install datasets
from datasets import Dataset  #Rôle : Importe la classe Dataset de Hugging Face,Utilité : Encapsule et gère vos données efficacement,Dataset pour structurer et manipuler efficacement les données avant l’entraînement.
#Importe la classe Dataset pour structurer et manipuler efficacement les données avant l’entraînement.
#Import des composants Transformers
#Importe les composants nécessaires pour:Tokenisation du texte, Classification binaire avec RoBERTa, Configuration et automatisation de l’entraînement
from transformers import (
    RobertaTokenizerFast,              # Convertit texte → nombres (version rapide ⚡)
    RobertaForSequenceClassification,  #Modèle RoBERTa pour classification binaire
    TrainingArguments,   #Configuration de l'entraînement (epochs, batch size, etc.)
    Trainer    #Automatise l'entraînement (forward, backward, optimisation)
)

# Charger le tokenizer (UNE SEULE FOIS)
#Charge le tokenizer RoBERTa pré-entraîné, qui convertit les URLs en tokens numériques exploitables par le modèle.
#Le tokenizer RoBERTa est entraîné sur du texte naturel → les URLs peuvent contenir beaucoup de caractères spéciaux.
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")  #Ce bloc charge le tokenizer RoBERTa, mais pas le modèle lui-même.
#RobertaTokenizerFast : La classe (version rapide)
#.from_pretrained() : Méthode de classe qui télécharge le tokenizer
#"roberta-base" : Nom du modèle sur Hugging Face Hub
#Résultat : Objet tokenizer prêt à convertir URLs en tokens
#Confirme le chargement correct du tokenizer et affiche la taille de son vocabulaire.
print("Tokenizer chargé avec succès!")
print(f"Taille du vocabulaire: {tokenizer.vocab_size}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.9/193.9 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.6/221.6 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 37.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.12.0
    Uninstalling fsspec-2025.12.0:
      Successfully uninstalled fsspec-2025.12.0


/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Tokenizer chargé avec succès!
Taille du vocabulaire: 50265


PyTorch est une bibliothèque de Deep Learning développée par Facebook AI (Meta). Permet de:
Créer et manipuler des tenseurs

Construire des réseaux de neurones

Faire de la rétropropagation automatique

Entraîner des modèles sur CPU ou GPU

Un tenseur = tableau multidimensionnel (comme une matrice NumPy mais optimisé pour GPU)

les bibliothèques transformers et datasets l'utilisent en coulisses pour :

Stocker les données sous forme de tenseurs
Entraîner le modèle sur GPU
Calculer les gradients

In [ ]:
#import torch # import PyTorch : bibliothèque principale pour le deep learning
#from datasets import Dataset
#from transformers import (
#    RobertaTokenizerFast, #Le tokenizer
 #   RobertaForSequenceClassification,#Le modèle RoBERTa prêt pour la classification.
  #  TrainingArguments,#Des réglages pour l’entraînement :nombre d’epochs (tours d’entraînement)+taille du batch
   # Trainer #Un assistant qui fait l’entraînement:calcule la loss+fait la rétropropagation+gère l’évaluation
#)

#tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base") #Télécharge et charge le tokenizer du modèle roberta-base”
#roberta-base = un modèle RoBERTa standard, déjà entraîné sur beaucoup de textes.
#C’est quoi Hugging Face ? un “App Store” de modèles d’IA :
#Tu peux télécharger des modèles déjà entraînés (comme RoBERTa, BERT, GPT…)
#Tu peux les utiliser directement pour prédire (sentiment, spam, etc.)
#Ou les ré-entraîner (“fine-tuning”) sur tes propres données
#Hugging Face fournit surtout 2 bibliothèques très utilisées :transformers : pour les modèles et tokenizers+datasets : pour gérer les jeux de données

Tokenisation (automatique)

In [ ]:
## ÉTAPE 2 : Définir la fonction de tokenisation
#but de cette fonction Transformer du texte en nombres +Un modèle comme RoBERTa ne lit pas du texte, il lit uniquement des nombres.
#Crée une fonction pour tokeniser un batch de données
def tokenize_batch(batch):
    return tokenizer(
        batch["url"],#le texte (URLs) à convertir en tokens
        truncation=True,  #Si le texte est trop long, on le coupe Ici, on ne garde que les 128 premiers tokens
        #coupe les textes trop longs pour ne garder que max_length tokens
        padding="max_length",  #padding="max_length" Tous les textes doivent avoir la même longueur, Les modèles aiment les tableaux bien alignés
        #remplit les textes courts jusqu’à max_length pour avoir des séquences de même taille
        max_length=128 #limite les séquences à 128 tokens
    )
    #La fonction tokenize_batch est appliquée par lots (batches) au dataset.Chaque batch contient une liste d’URLs, qui sont simultanément tokenisées par le tokenizer RoBERTa.
#Cette approche est plus efficace que la tokenisation individuelle et est requise par l’API HuggingFace.

    #Cette fonction :prend du texte (url)+le transforme en nombres+avec une taille fixe (128)+Elle est obligatoire avant d’entraîner un modèle RoBERTa

3 DataFrames pandas :

Tu veux :

Les convertir en HuggingFace Dataset

Appliquer le tokenizer RoBERTa

Obtenir des datasets prêts pour l’entraînement

ÉTAPE 1 — Conversion Pandas → HuggingFace Dataset

RoBERTa + Trainer ne travaillent pas directement avec pandas.

In [ ]:
#Conversion des datasets Pandas
#Transforme chaque DataFrame Pandas en un dataset Hugging Face compatible avec Trainer et Dataset.map().

hf_balanced = Dataset.from_pandas(dataset_balanced)
hf_90_10 = Dataset.from_pandas(dataset_imb_90_10)
hf_10_90 = Dataset.from_pandas(dataset_imb_10_90)

ÉTAPE 2 — Appliquer le tokenizer aux 3 datasets

Tokenisation du dataset équilibré (50–50)

In [ ]:
# ÉTAPE 3 : Appliquer sur le dataset
hf_balanced = hf_balanced.map(
    tokenize_batch,#Applique la fonction tokenize_batch en batch sur le dataset Hugging Face.
    batched=True,#Rôle : Traiter les données par lots (ex: 1000 lignes à la fois) au lieu d'une par une Taille du batch par défaut : 1000 lignes/ traitement vectorisé, plus rapide pour les grands datasets.
    remove_columns=["url"]#Rôle : Supprime la colonne "url" après tokenisation// Supprime la colonne "url" après tokenisation, car le modèle n’a besoin que des input_ids et attention_mask.
    #La colonne "url" (texte) n'est plus nécessaire,✅ Le modèle n'utilise que input_ids et attention_mask,Économise de la mémoire

)
#Taille du batch non précisée → Hugging Face utilise 1000 par défaut, mais peut être ajustée si GPU limité.

Map:   0%|          | 0/176882 [00:00<?, ? examples/s]

# AVANT .map()
hf_balanced = {
    'url': ['google.com', 'evil.com', ...],      

    'label': [0, 1, ...]
}

# APRÈS .map() avec remove_columns=["url"]
hf_balanced = {
    'label': [0, 1, ...],                         

    'input_ids': [[0, 16357, ...], ...],          

    'attention_mask': [[1, 1, ...], ...]          
    
    # 'url' est supprimé ✂️
}

Tokenisation du dataset 90–10

In [ ]:
#Application sur les datasets déséquilibrés
hf_90_10 = hf_90_10.map(
    tokenize_batch,
    batched=True,
    remove_columns=["url"]
)


Map:   0%|          | 0/176882 [00:00<?, ? examples/s]

Tokenisation du dataset 10–90

In [ ]:

#Application sur les datasets déséquilibrés
hf_10_90 = hf_10_90.map(
    tokenize_batch,
    batched=True,
    remove_columns=["url"]
)

Map:   0%|          | 0/176882 [00:00<?, ? examples/s]

#ÉTAPE 3 — Définir le format PyTorch (OBLIGATOIRE)

RoBERTa lit uniquement des tensors PyTorch.

In [ ]:
#Définition des colonnes à utiliser
#Spécifie les colonnes que le modèle RoBERTa utilisera pour l’entraînement :
#input_ids : séquences de tokens
#attention_mask : masque pour indiquer les tokens à considérer
#label : la cible binaire (0 ou 1)
columns = ["input_ids", "attention_mask", "label"]
#conversion au format PyTorch   Convertit les datasets HF en tenseurs PyTorch, prêts pour l’entraînement. Ne garde que les colonnes spécifiées (input_ids, attention_mask, label) → optimisation mémoire.
#Convertit les datasets HF en tenseurs PyTorch, prêts pour l’entraînement.
hf_balanced.set_format(type="torch", columns=columns)
hf_90_10.set_format(type="torch", columns=columns)
hf_10_90.set_format(type="torch", columns=columns)


Vérification rapide (optionnelle mais recommandée)

In [ ]:
print(hf_balanced)
print(hf_90_10)
print(hf_10_90)

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 176882
})
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 176882
})
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 176882
})


Les trois distributions (50–50, 90–10, 10–90) sont converties en datasets HuggingFace. Le tokenizer RoBERTa pré-entraîné (roberta-base) est appliqué par lots à l’ensemble des URLs afin de transformer le texte en représentations numériques (input_ids, attention_mask). Les datasets tokenisés sont ensuite formatés en tenseurs PyTorch pour être utilisés lors de l’entraînement.

Séparation des données PHASE 4 – SPLIT TRAIN / TEST (80%-20%)

In [ ]:
#Split train/test
#Exemple pour dataset équilibré
#Sépare le dataset Hugging Face en train (80%) et test (20%)
#seed=42 → reproductibilité du split. Retourne un dictionnaire avec train et test datasets.
split_balanced = hf_balanced.train_test_split(test_size=0.2, seed=42)
train_balanced = split_balanced["train"]
test_balanced = split_balanced["test"]

# Vérification
#Affiche le nombre d’échantillons pour train et test afin de vérifier le split.
print("Balanced Dataset")
print("Train size:", len(train_balanced))
print("Test size :", len(test_balanced))

#Vérification de la distribution des classes , compte les occurrences de chaque classe dans train et test.
#Vérifie si le split préserve l’équilibre du dataset.
print("Train class distribution:\n", list(train_balanced['label']).count(0), "benign,", list(train_balanced['label']).count(1), "malicious")
print("Test class distribution:\n", list(test_balanced['label']).count(0), "benign,", list(test_balanced['label']).count(1), "malicious")

Balanced Dataset
Train size: 141505
Test size : 35377
Train class distribution:
 70803 benign, 70702 malicious
Test class distribution:
 17638 benign, 17739 malicious



Répéter pour les 2 autres datasets

In [ ]:
# 90%-10%
split_90_10 = hf_90_10.train_test_split(test_size=0.2, seed=42)
train_90_10 = split_90_10["train"]
test_90_10 = split_90_10["test"]
#Sépare le dataset 90%-10% en train (80%) et test (20%). seed=42 → reproductible. Retourne un dictionnaire avec train et test datasets.

# 10%-90%
split_10_90 = hf_10_90.train_test_split(test_size=0.2, seed=42)
train_10_90 = split_10_90["train"]
test_10_90 = split_10_90["test"]
#Vérification similaire pour chacun (taille + distribution)

PHASE 5 – FINE-TUNING AVEC LoRA Préparer le modèle

In [ ]:
from transformers import RobertaForSequenceClassification #modèle RoBERTa pour classification binaire.
from peft import LoraConfig, get_peft_model #outils pour appliquer LoRA (Low-Rank Adaptation) afin d’adapter le modèle avec moins de paramètres entraînables.
#Ici, tu charges le modèle RoBERTa complet pour classification, pas juste le tokenizer.
#Le tokenizer et le modèle sont deux objets différents :

#Tokenizer → convertit le texte en tokens (input_ids, attention_mask)

#Modèle → reçoit ces tokens et produit des prédictions (logits, probas, etc.)
#Donc oui, le tokenizer était déjà téléchargé, mais le modèle lui-même n’était pas encore chargé, donc cette étape est nécessaire avant le fine-tuning avec LoRA.
# Charger modèle pré-entraîné
model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base",     #Ici, tu charges le modèle RoBERTa complet pour classification, pas juste le tokenizer.
    num_labels=2 #Télécharge le modèle RoBERTa pré-entraîné. Définit num_labels=2 pour la classification binaire (benign/malicious).
)
#la taille du modèle est grande (~125M paramètres) → GPU nécessaire pour fine-tuning complet.
#💡 LoRA réduit ce coût en ne modifiant que des sous-blocs.
# Configuration LoRA Config LoRA
#Paramètres standard pour fine-tuning efficace avec LoRA
lora_config = LoraConfig(
    r=8, #rang de la décomposition low-rank → contrôle la capacité d’adaptation
    lora_alpha=16,   #facteur d’échelle → stabilise l’entraînement
    lora_dropout=0.1,  #régularisation pour éviter l’overfitting
    target_modules=["query", "value"], #appliqué uniquement aux matrices Q et V des transformers
    bias="none",
    task_type="SEQ_CLS"  # Sequence Classification, tâche de classification de séquences
)

# Appliquer LoRA  , Appliquer LoRA au modèle
model = get_peft_model(model, lora_config)
#Transforme le modèle RoBERTa en modèle LoRA, où seules certaines couches sont entraînables.

#Réduit la mémoire GPU nécessaire et accélère le fine-tuning.

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Préparer l’entraînement

In [ ]:
from transformers import TrainingArguments, Trainer
# Arguments d’entraînement (TrainingArguments)\\\\\\TrainingArguments : classe qui permet de définir tous les paramètres d’entraînement du modèle.
#argument d entranment :Permettent de configurer entièrement le training :Nombre d’epochs, taille des batches, learning rate, Stratégie d’évaluation et de sauvegarde GPU/FP16, logging, etc.

C’est le “cœur” qui contrôle comment le modèle s’entraîne.
#Trainer : classe qui gère automatiquement l’entraînement, l’évaluation et la sauvegarde du modèle. Créer le Trainer Trainer automatisera tout l’entraînement : forward/backward pass, calcul de loss, optimisation, scheduler, checkpointing.
#Trainer appelle automatiquement  fonction compte metric à chaque évaluation (eval_dataset).
#On lui fournit : model → le modèle à entraîner (ici avec LoRA), train_dataset et eval_dataset, compute_metrics → pour calculer précision, recall, F1 à chaque évaluation
training_args = TrainingArguments(
    output_dir="./roberta_lora_results", ## dossier pour sauvegarder les checkpoints
    num_train_epochs=1, #nombre de passages sur tout le dataset
    per_device_train_batch_size=16, # # batch size pour l'entraînement
    per_device_eval_batch_size=32,## batch size pour évaluation
    learning_rate=5e-5,
    save_strategy="epoch", # sauvegarder modèle à chaque epoch
    eval_strategy="epoch", # Added evaluation strategy # évaluer à chaque fin d'epoch
    load_best_model_at_end=True, ## charger le meilleur modèle à la fin
    fp16=True  # # activer float16 si GPU compatible
)


Fonction d’évaluation

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

In [ ]:
# 3️⃣ Définir la fonction de métriques
#calculer des métriques personnalisées (precision, recall, F1-score) pendant l’évaluation.
def compute_metrics(eval_pred):
    """
    Cette fonction est appelée par Trainer à chaque évaluation.
    eval_pred: tuple (logits, labels)
    Retourne: dictionnaire {"precision":..., "recall":..., "f1":...}
    """
    #Entrée : un tuple (logits, labels), Sortie : dictionnaire avec les métriques, compatible avec HF Trainer.
    logits, labels = eval_pred
    # Convertir logits en prédictions de classe
    preds = torch.argmax(torch.tensor(logits), axis=1).numpy()
    labels = labels.astype(int)

    # Calcul des métriques
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

Fine-tuning pour les 3 datasets Utilisation avec Trainer

In [ ]:
#Créer Trainer
trainer_balanced = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_balanced, ##  avec dataset équilibré
    eval_dataset=test_balanced,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
# Lancer l'entraînement
trainer_balanced.train() ## entraîne le modèle en appliquant LoRA sur les couches ciblées


/tmp/ipython-input-2023436665.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_balanced = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.029200,0.037436,0.998860,0.987711,0.993254


TrainOutput(global_step=8845, training_loss=0.04694614938721945, metrics={'train_runtime': 806.7781, 'train_samples_per_second': 175.395, 'train_steps_per_second': 10.963, 'total_flos': 9404282506644480.0, 'train_loss': 0.04694614938721945, 'epoch': 1.0})

In [ ]:
# 7️⃣ Évaluation finale
# =======================
#évalue le modèle sur le dataset de validation/test. Retourne un dictionnaire avec loss et métriques (ici precision, recall, F1).

Permet de comparer les performances pour les différents datasets (équilibré ou déséquilibré).
metrics = trainer.evaluate()  # évalue le modèle sur le dataset de test
#evaluate() est une méthode fournie par la classe Trainer de Hugging Face.
#Trainer fait automatiquement : Passe le modèle en mode évaluation (model.eval())
#Parcourt eval_dataset
#Calcule :  Validation loss (automatique), Les métriques définies dans compute_metrics, Retourne un dictionnaire de résultats
print("Résultats finaux :")
print(metrics)

In [ ]:
#afficher les résultats de precision, recall et F1-score après l’évaluation

 réutiliser le même Trainer en changeant simplement :

train_dataset=train_90_10
eval_dataset=test_90_10


ou

train_dataset=train_10_90
eval_dataset=test_10_90

In [ ]:
trainer_90_10 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_90_10,
    eval_dataset=test_90_10,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer_90_10.train()


/tmp/ipython-input-1243474842.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_90_10 = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.015400,0.014370,0.995223,0.981435,0.988281


TrainOutput(global_step=8845, training_loss=0.012361531961159628, metrics={'train_runtime': 770.383, 'train_samples_per_second': 183.681, 'train_steps_per_second': 11.481, 'total_flos': 9404282506644480.0, 'train_loss': 0.012361531961159628, 'epoch': 1.0})

In [ ]:
trainer_10_90 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_10_90,
    eval_dataset=test_10_90,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer_10_90.train()

/tmp/ipython-input-3888788233.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_10_90 = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.018900,0.024938,0.999431,0.993282,0.996347


TrainOutput(global_step=8845, training_loss=0.02050039130728955, metrics={'train_runtime': 772.571, 'train_samples_per_second': 183.161, 'train_steps_per_second': 11.449, 'total_flos': 9404282506644480.0, 'train_loss': 0.02050039130728955, 'epoch': 1.0})

Points importants à retenir

Même modèle + mêmes hyperparamètres pour les 3 distributions
→ Permet une comparaison juste

LoRA adapte seulement certaines couches (query, value)
→ Très efficace pour fine-tuning avec peu de ressources

Train/Test split 80/20 est appliqué avant LoRA

Vérification de la distribution des classes
→ Toujours nécessaire pour détecter les biais

In [ ]:
#SMOTE / SMOTE-Tomek NE s’appliquent PAS directement sur du texte
#👉 Ils s’appliquent sur des vecteurs numériques

#Dans ton pipeline avec RoBERTa, il y a 2 façons correctes :

#✅ Approche acceptable en recherche (la plus utilisée)

#Appliquer SMOTE / SMOTE-Tomek uniquement sur le TRAIN set

#Utiliser les features numériques (embeddings / input_ids)

#Tester ensuite sur le test set original (non rééchantillonné)

In [ ]:
#Afin de traiter le déséquilibre de classes, les techniques SMOTE et SMOTE-Tomek ont été appliquées uniquement sur les ensembles d’entraînement des distributions 90–10 et 10–90.
#Les performances ont été évaluées à l’aide des métriques precision, recall et F1-score sur les ensembles de test non rééchantillonnés.

In [ ]:
#La meilleure option : utiliser les embeddings (pas les input_ids)
#👉 Les input_ids sont la moins bonne représentation pour SMOTE

In [ ]:
Les techniques de rééchantillonnage (SMOTE et SMOTE-Tomek) ont été appliquées sur les représentations vectorielles continues extraites de RoBERTa (embedding du token [CLS]),
 plutôt que sur les indices discrets des tokens, afin de garantir la cohérence géométrique de l’espace de features.

In [ ]:
Les techniques SMOTE et SMOTE-Tomek ont été appliquées sur les représentations vectorielles continues extraites du token CLS de RoBERTa. Cette approche permet de préserver la structure géométrique
de l’espace de features et d’améliorer la détection de la classe minoritaire.